In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression                                       #linear regression
from sklearn.preprocessing import PolynomialFeatures                                    #polynomial regression
from sklearn.svm import SVR                                                             #simple vector regression
from sklearn.tree import DecisionTreeRegressor                                          #decision tree regressor
from sklearn.ensemble import RandomForestRegressor                                      #random forest regressor
import gurobipy as gp
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

In [20]:
%%capture
dataset = pd.read_csv('input/adatb_setcover_output.csv')
for i in range(5000):
    m = gp.read("./train/instance_"+str(i+1)+".lp")                          # reading the LP files
    A = m.getA()                                                             # creating sparse matrix from LP files
    B=A.toarray()                                                            # converting the sparse matrix to ndarray
    dataset.at[i,"instance"]=B                                               # writing this matrix to the dataframe
    
#dropping columns that seem irrelevant

dataset = dataset.drop('policy', axis=1)
dataset = dataset.drop('seed', axis=1)
dataset = dataset.drop('type', axis=1)
dataset = dataset.drop('status', axis=1)
dataset = dataset.drop('nnodes', axis=1)
dataset = dataset.drop('nlps', axis=1)
dataset = dataset.drop('gap', axis=1)
dataset = dataset.drop('ndomchgs', axis=1)
dataset = dataset.drop('ncutoffs', axis=1)

m = gp.read("./train/instance_2.lp")                          # reading the LP files
l=[]
for i in range(5000):
    m = gp.read("./train/instance_"+str(i+1)+".lp")
    c = m.getAttr("Obj",m.getVars())
    l=l+[c]


In [21]:
%%capture
m = gp.read("./train/instance_2.lp")                          # reading the LP files
l=[]
for i in range(5000):
    m = gp.read("./train/instance_"+str(i+1)+".lp")
    c = m.getAttr("Obj",m.getVars())
    l=l+[c]
dataset["objective"]=l

In [22]:
dataset.head()

,instance,stime,walltime,proctime,objval,objective
0,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.03,0.041468,0.041470,28.0,"[19.0, 91.0, 40.0, 51.0, 78.0, 3.0, 84.0, 78.0..."
1,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,...",0.12,0.117496,0.117499,25.0,"[67.0, 4.0, 48.0, 73.0, 9.0, 99.0, 27.0, 61.0,..."
2,"[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.03,0.032750,0.032753,10.0,"[22.0, 48.0, 71.0, 56.0, 16.0, 24.0, 17.0, 44...."
3,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0,...",0.05,0.046181,0.046185,23.0,"[31.0, 88.0, 70.0, 27.0, 76.0, 45.0, 29.0, 37...."
4,"[[0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0,...",0.01,0.014738,0.014741,11.0,"[62.0, 4.0, 53.0, 1.0, 46.0, 24.0, 50.0, 72.0,..."


In [15]:
dataset_flat=pd.read_csv("flattened_dataset.csv")
#dataset_flat["nlps"]=dataset["nlps"]
dataset_flat.head(10)

,Unnamed: 0,stime,walltime,proctime,objval,a0,a1,a2,a3,a4,...,c191,c192,c193,c194,c195,c196,c197,c198,c199,nlps
0,0,0.03,0.041468,0.041470,28.0,1.0,0.0,0.0,0.0,0.0,...,80.0,28.0,54.0,30.0,31.0,16.0,88.0,47.0,32.0,2
1,1,0.12,0.117496,0.117499,25.0,1.0,0.0,0.0,0.0,0.0,...,28.0,17.0,98.0,18.0,79.0,49.0,2.0,50.0,1.0,8
2,2,0.03,0.032750,0.032753,10.0,0.0,0.0,1.0,0.0,0.0,...,38.0,30.0,59.0,96.0,33.0,29.0,81.0,82.0,85.0,3
3,3,0.05,0.046181,0.046185,23.0,0.0,1.0,0.0,0.0,0.0,...,75.0,30.0,92.0,17.0,20.0,10.0,19.0,95.0,28.0,2
4,4,0.01,0.014738,0.014741,11.0,0.0,0.0,1.0,1.0,0.0,...,78.0,34.0,53.0,44.0,67.0,86.0,1.0,16.0,60.0,1
5,5,0.24,0.250231,0.250223,17.0,1.0,0.0,0.0,0.0,0.0,...,62.0,62.0,41.0,66.0,75.0,8.0,94.0,15.0,55.0,26
6,6,0.33,0.331593,0.331595,19.0,1.0,0.0,0.0,1.0,0.0,...,61.0,28.0,82.0,6.0,77.0,14.0,16.0,93.0,39.0,30
7,7,0.35,0.367199,0.367201,30.0,0.0,1.0,0.0,1.0,1.0,...,55.0,85.0,24.0,80.0,31.0,71.0,13.0,2.0,88.0,37
8,8,0.02,0.017138,0.017119,10.0,1.0,0.0,0.0,0.0,1.0,...,61.0,29.0,36.0,5.0,61.0,63.0,46.0,38.0,43.0,2
9,9,0.02,0.015780,0.015783,32.0,0.0,1.0,0.0,0.0,0.0,...,39.0,89.0,19.0,66.0,26.0,3.0,15.0,63.0,54.0,1


In [9]:
X_train = dataset_flat
X_train = X_train.drop('stime', axis=1)
X_train = X_train.drop('walltime', axis=1)
X_train = X_train.drop('proctime', axis=1)
X_train = X_train.drop('objval', axis=1)
X_train = X_train.drop('nlps', axis=1)
X_train = X_train.drop(X_train.columns[0], axis=1)
X_test = X_train[4500:]
X_train = X_train[:4500]
Y_train = dataset_flat['proctime']
Y_test = Y_train[4500:]
Y_train = Y_train[:4500]
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(4500, 20200) (4500,) (500, 20200) (500,)


In [24]:
# statistics for proctime and nlps
time_mean = np.mean(dataset_flat["proctime"])
time_var = np.var(dataset_flat["proctime"])
nlps_mean = np.mean(dataset_flat["nlps"])
nlps_var = np.var(dataset_flat["nlps"])
print("Runtime mean: ", time_mean)
print("Runtime variance: ", time_var)
print("Number of LPs mean: ", nlps_mean)
print("Number of LPs variance: ", nlps_var)

Runtime mean:  0.1398399791612002
Runtime variance:  0.021164107353354448
Number of LPs mean:  12.6888
Number of LPs variance:  238.74075456000003


In [10]:
#linear regression for proctime
regression_model = LinearRegression()
regression_model = regression_model.fit(X_train,Y_train)
pred=regression_model.predict(X_test)
print("Mean squared error for linear regression:", mean_squared_error(Y_test, pred))
print("Mean absolute error for linear regression: ", str(mean_absolute_error(pred, Y_test)))

Mean squared error for linear regression: 0.023387391563361577
Mean absolute error for linear regression:  0.12330516355637705


In [11]:
#xgboost for proctime
my_model = XGBRegressor(nthread=10)
my_model.fit(X_train, Y_train, verbose=False)
predictions = my_model.predict(X_test)
print("Mean Squared Error for XGBoost: "+str(mean_squared_error(Y_test, predictions)))
print("Mean Absolute Error for XGBoost: " + str(mean_absolute_error(predictions, Y_test)))

/home/dankod/miniconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Mean Squared Error for XGBoost: 0.020596165487422154
Mean Absolute Error for XGBoost: 0.1153811013540053


In [12]:
# linear regression for nlps
Y_train = dataset_flat['nlps']
Y_test = Y_train[4500:]
Y_train = Y_train[:4500]
regression_model = LinearRegression()
regression_model = regression_model.fit(X_train,Y_train)
pred=regression_model.predict(X_test)
print("Mean squared error for linear regression:", mean_squared_error(Y_test, pred))
print("Mean absolute error for linear regression: ", str(mean_absolute_error(pred, Y_test)))

Mean squared error for linear regression: 271.98231873596984
Mean absolute error for linear regression:  13.212698152087167


In [13]:
# xgBoost for nlps
my_model = XGBRegressor(nthread=10)
my_model.fit(X_train, Y_train, verbose=False)
predictions = my_model.predict(X_test)
print("Mean Squared Error for XGBoost: "+str(mean_squared_error(Y_test, predictions)))
print("Mean Absolute Error for XGBoost: " + str(mean_absolute_error(predictions, Y_test)))

/home/dankod/miniconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Mean Squared Error for XGBoost: 241.16588705524512
Mean Absolute Error for XGBoost: 12.648571887508036


In [16]:
np.mean(dataset_flat['nlps'])

12.6888